In [ ]:
from google.colab import drive
import nbformat, os

drive.mount('/content/drive', force_remount=True)
SRC = '/content/drive/MyDrive/Colab Notebooks/Untitled11.ipynb'
DST = '/content/drive/MyDrive/colab_saves/tune last 3 layers.ipynb'

with open(SRC, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)


nb.metadata.pop('widgets', None)
for cell in nb.cells:
    cell.metadata.pop('widgets', None)
os.makedirs(os.path.dirname(DST), exist_ok=True)
with open(DST, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("✅ Clean copy saved to", DST)

In [ ]:
!pip install datasets sentence-transformers
!pip install --upgrade datasets

In [1]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample
import torch
import os, time, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from datasets import load_dataset
from torch.utils.data import DataLoader
from sentence_transformers.losses import CosineSimilarityLoss
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import os, time, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import numpy as np
import pickle
from google.colab import drive

In [2]:
import os, time, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

API_KEY = os.getenv("RAPIDAPI_KEY") or "ad973560c8msh96909b7fb3cc3fdp1c1065jsn5a49e6b93474"
HEADERS = {
    "X-RapidAPI-Key":  API_KEY,
    "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com",
}

session = requests.Session()
retry_cfg = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"],
)
session.mount("https://", HTTPAdapter(max_retries=retry_cfg))


def _respect_rate_limit(resp):
    """Sleep if remaining-requests header is low."""
    left = resp.headers.get("X-RateLimit-Requests-Remaining")
    reset = resp.headers.get("X-RateLimit-Requests-Reset")
    if left is not None and reset is not None:
        left, reset = int(left), int(reset)
        if left < 2:
            sleep_s = max(reset - int(time.time()), 1)
            print(f"[rate-limit] only {left} calls left → sleeping {sleep_s}s")
            time.sleep(sleep_s)

# extracting the news with particular symbols

def fetch_symbol_news(symbol: str,
                      max_items: int = 200,
                      page_size: int = 40,
                      since: int = 0,
                      until: int = 0) -> pd.DataFrame:
    url   = "https://seeking-alpha.p.rapidapi.com/news/v2/list-by-symbol"
    items = []
    page  = 1

    while len(items) < max_items:
        p = {"id": symbol, "size": page_size, "number": page}
        if since: p["since"] = since
        if until: p["until"] = until

        r = session.get(url, headers=HEADERS, params=p, timeout=20)
        r.raise_for_status()
        batch = r.json().get("data", [])
        if not batch:
            break

        items.extend(batch)
        if len(batch) < page_size:
            break

        page += 1
        time.sleep(0.25 + 0.25 * os.urandom(1)[0] / 255)

    if not items:
        return pd.DataFrame(columns=["title", "publishOn", "symbol"])

    df = pd.json_normalize(items[:max_items])

    for c in ["attributes.title", "attributes.headline", "attributes.teaser"]:
        if c in df.columns:
            df = df.rename(columns={c: "title"})
            break
    if "title" not in df.columns:
        df["title"] = ""

    if "publishOn" in df.columns:
        df["publishOn"] = pd.to_datetime(df["publishOn"], utc=True)
    elif "publishedAt" in df.columns:
        df["publishOn"] = pd.to_datetime(df["publishedAt"], utc=True)

    df["symbol"] = symbol
    return df



In [3]:
# collect the return data on the publish day of news and convert to 1,0
import yfinance as yf
def label_news_df(df: pd.DataFrame, symbol: str) -> pd.DataFrame:
    df = df.copy()
    if "attributes.publishOn" in df.columns:
        df = df.rename(columns={"attributes.publishOn": "publishOn"})


    df["publishOn"] = (
        pd.to_datetime(df["publishOn"], utc=True, errors="coerce")
          .dt.tz_localize(None)
    )
    df = df.dropna(subset=["publishOn"])

    # 1) build price window
    earliest = df["publishOn"].dt.date.min()
    latest   = df["publishOn"].dt.date.max()
    start_dt = (pd.Timestamp(earliest) - pd.Timedelta(days=7)).strftime("%Y-%m-%d")
    end_dt   = (pd.Timestamp(latest)   + pd.Timedelta(days=7)).strftime("%Y-%m-%d")

    px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")

    # extract the Close price
    closes = px["Close"]
    if isinstance(closes, pd.DataFrame):
        closes = closes.squeeze()

    # normalise to midnight dates
    closes.index = closes.index.tz_localize(None).normalize()
    trading_dates = closes.index

    ret_vals, labels = [], []
    for ts in df["publishOn"]:
        pub = pd.to_datetime(ts).tz_localize(None).normalize()

        prev_days = trading_dates[trading_dates < pub]
        next_days = trading_dates[trading_dates > pub]
        if prev_days.empty or next_days.empty:
            ret_vals.append(np.nan)
            labels.append(np.nan)
            continue

        d0, d1 = prev_days.max(), next_days.min()
        p0, p1 = closes.at[d0], closes.at[d1]

        if pd.isna(p0) or pd.isna(p1) or p0 == 0:
            ret_vals.append(np.nan)
            labels.append(np.nan)
            continue

        r = p1 / p0 - 1
        ret_vals.append(r)
        labels.append(int(r > 0))

    df["return_1d"] = ret_vals
    df["label"]     = labels
    df = df.dropna(subset=["label"]).reset_index(drop=True)
    return df


In [4]:

drive.mount('/content/drive', force_remount=True)

path = '/content/drive/My Drive/colab_saves/company_dfs.pkl'
with open(path, 'rb') as f:
    company_dfs = pickle.load(f)

company_df_2 = company_dfs['group2']
company_df_3 = company_dfs['group3']
company_df_4 = company_dfs['group4']

Mounted at /content/drive


In [5]:

import numpy as np

labeled_group2 = {
    sym: label_news_df(df, sym)
    for sym, df in company_df_2.items()
}

labeled_group3 = {
    sym: label_news_df(df, sym)
    for sym, df in company_df_3.items()
}

labeled_group4 = {
    sym: label_news_df(df, sym)
    for sym, df in company_df_4.items()
}


/tmp/ipython-input-3951426405.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3951426405.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3951426405.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3951426405.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")
[*********************100%***********************]  1

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for p in model.parameters():
    p.requires_grad = False

transformer = model._modules['0']
hf_model    = transformer.auto_model

emb = hf_model.get_input_embeddings()
for p in emb.parameters():
    p.requires_grad = True

blocks = None
for name, module in hf_model.named_modules():
    if isinstance(module, torch.nn.ModuleList):
        blocks = module
        print(f"→ Found blocks under: {name}")
        break

if blocks is None:
    raise RuntimeError("Could not find transformer blocks in hf_model")

# unfreeze last 3 blocks
for block in blocks[-3:]:
    for p in block.parameters():
        p.requires_grad = True

print(f"Un-froze {sum(p.requires_grad for p in model.parameters())} params total.")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

→ Found blocks under: layers
Un-froze 34 params total.


In [7]:


results = {}

for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"=== Tuning on {group_name} ===")
    df_all = pd.concat(labeled_group.values(), ignore_index=True)
    texts = df_all["title"].tolist()
    labels = df_all["label"].astype(int).tolist()
    X_train, X_test, y_train, y_test = train_test_split(
        texts, labels, test_size=0.2, random_state=42, stratify=labels
    )

    emb_train = model.encode(
        X_train, device=device, batch_size=16,
        convert_to_tensor=True, show_progress_bar=True
    )
    del X_train
    emb_test = model.encode(
        X_test, device=device, batch_size=16,
        convert_to_tensor=True, show_progress_bar=True
    )
    del X_test

    d = emb_train.size(1)
    head = nn.Sequential(
        nn.Linear(d, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    ).to(device)

    trainable = list(head.parameters()) + [
        p for p in model.parameters() if p.requires_grad
    ]
    optimizer = torch.optim.Adam(trainable, lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    train_ds = TensorDataset(emb_train, torch.tensor(y_train, device=device))
    train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)

    best_prev_loss = float("inf")
    tol = 1e-4
    patience = 3
    stale_epochs = 0
    max_epochs = 100

    for epoch in range(1, max_epochs + 1):
        head.train()
        epoch_loss = 0.0
        for emb_b, lab_b in train_dl:
            optimizer.zero_grad()
            logits = head(emb_b)
            loss = criterion(logits, lab_b)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * emb_b.size(0)
        avg_loss = epoch_loss / len(train_ds)
        print(f"  Epoch {epoch:>3} — Loss: {avg_loss:.6f}")
        improvement = best_prev_loss - avg_loss
        if improvement < tol:
            stale_epochs += 1
            print(f"  ↳ small improvement (Δ={improvement:.2e}) stale {stale_epochs}/{patience}")
        else:
            stale_epochs = 0
            best_prev_loss = avg_loss
        if stale_epochs >= patience:
            print(f"  ↳ Early stopping at epoch {epoch}")
            break

    head.eval()
    with torch.no_grad():
        logits = head(emb_test.to(device))
        preds = logits.argmax(dim=1).cpu().tolist()

    acc = accuracy_score(y_test, preds)
    micro_f1 = f1_score(y_test, preds, average="micro")
    macro_f1 = f1_score(y_test, preds, average="macro")
    weighted_f1 = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")

    print(f"Metrics for {group_name}:")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")

    results[group_name] = head



=== Tuning on group2 ===


Batches:   0%|          | 0/80 [00:00<?, ?it/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

  Epoch   1 — Loss: 0.695294
  Epoch   2 — Loss: 0.690311
  Epoch   3 — Loss: 0.686781
  Epoch   4 — Loss: 0.683695
  Epoch   5 — Loss: 0.680532
  Epoch   6 — Loss: 0.677432
  Epoch   7 — Loss: 0.673674
  Epoch   8 — Loss: 0.670924
  Epoch   9 — Loss: 0.667297
  Epoch  10 — Loss: 0.663321
  Epoch  11 — Loss: 0.659341
  Epoch  12 — Loss: 0.656053
  Epoch  13 — Loss: 0.652369
  Epoch  14 — Loss: 0.649097
  Epoch  15 — Loss: 0.646523
  Epoch  16 — Loss: 0.643264
  Epoch  17 — Loss: 0.640164
  Epoch  18 — Loss: 0.637079
  Epoch  19 — Loss: 0.634835
  Epoch  20 — Loss: 0.632097
  Epoch  21 — Loss: 0.628944
  Epoch  22 — Loss: 0.627141
  Epoch  23 — Loss: 0.624293
  Epoch  24 — Loss: 0.622324
  Epoch  25 — Loss: 0.619247
  Epoch  26 — Loss: 0.617194
  Epoch  27 — Loss: 0.616070
  Epoch  28 — Loss: 0.613386
  Epoch  29 — Loss: 0.610550
  Epoch  30 — Loss: 0.608573
  Epoch  31 — Loss: 0.606734
  Epoch  32 — Loss: 0.604313
  Epoch  33 — Loss: 0.603007
  Epoch  34 — Loss: 0.600166
  Epoch  35 — 

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

  Epoch   1 — Loss: 0.687900
  Epoch   2 — Loss: 0.685328
  Epoch   3 — Loss: 0.683175
  Epoch   4 — Loss: 0.681184
  Epoch   5 — Loss: 0.678728
  Epoch   6 — Loss: 0.676334
  Epoch   7 — Loss: 0.673273
  Epoch   8 — Loss: 0.669825
  Epoch   9 — Loss: 0.666936
  Epoch  10 — Loss: 0.663457
  Epoch  11 — Loss: 0.660308
  Epoch  12 — Loss: 0.656733
  Epoch  13 — Loss: 0.653184
  Epoch  14 — Loss: 0.650052
  Epoch  15 — Loss: 0.646600
  Epoch  16 — Loss: 0.643048
  Epoch  17 — Loss: 0.639781
  Epoch  18 — Loss: 0.636634
  Epoch  19 — Loss: 0.632952
  Epoch  20 — Loss: 0.629421
  Epoch  21 — Loss: 0.626524
  Epoch  22 — Loss: 0.622782
  Epoch  23 — Loss: 0.619146
  Epoch  24 — Loss: 0.616555
  Epoch  25 — Loss: 0.613369
  Epoch  26 — Loss: 0.611466
  Epoch  27 — Loss: 0.607057
  Epoch  28 — Loss: 0.603855
  Epoch  29 — Loss: 0.601269
  Epoch  30 — Loss: 0.597699
  Epoch  31 — Loss: 0.595580
  Epoch  32 — Loss: 0.592054
  Epoch  33 — Loss: 0.589703
  Epoch  34 — Loss: 0.586998
  Epoch  35 — 

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

  Epoch   1 — Loss: 0.691186
  Epoch   2 — Loss: 0.686917
  Epoch   3 — Loss: 0.684729
  Epoch   4 — Loss: 0.683142
  Epoch   5 — Loss: 0.681488
  Epoch   6 — Loss: 0.679624
  Epoch   7 — Loss: 0.677680
  Epoch   8 — Loss: 0.675755
  Epoch   9 — Loss: 0.673161
  Epoch  10 — Loss: 0.670975
  Epoch  11 — Loss: 0.668877
  Epoch  12 — Loss: 0.666188
  Epoch  13 — Loss: 0.663342
  Epoch  14 — Loss: 0.661270
  Epoch  15 — Loss: 0.658217
  Epoch  16 — Loss: 0.655543
  Epoch  17 — Loss: 0.652600
  Epoch  18 — Loss: 0.650092
  Epoch  19 — Loss: 0.647826
  Epoch  20 — Loss: 0.644532
  Epoch  21 — Loss: 0.642669
  Epoch  22 — Loss: 0.639380
  Epoch  23 — Loss: 0.636801
  Epoch  24 — Loss: 0.633805
  Epoch  25 — Loss: 0.631374
  Epoch  26 — Loss: 0.629801
  Epoch  27 — Loss: 0.626540
  Epoch  28 — Loss: 0.623956
  Epoch  29 — Loss: 0.621356
  Epoch  30 — Loss: 0.619273
  Epoch  31 — Loss: 0.616577
  Epoch  32 — Loss: 0.614549
  Epoch  33 — Loss: 0.612039
  Epoch  34 — Loss: 0.609169
  Epoch  35 — 

In [8]:

for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"=== Recomputing embeddings for {group_name} ===")
    df_all = pd.concat(labeled_group.values(), ignore_index=True)
    texts = df_all["title"].tolist()
    labels = df_all["label"].astype(int).tolist()
    _, X_test, _, y_test = train_test_split(
        texts, labels,
        test_size=0.2,
        random_state=42,
        stratify=labels
    )

    emb_test_updated = model.encode(
        X_test,
        device=device,
        batch_size=16,
        convert_to_tensor=True,
        show_progress_bar=True
    )

    head = results[group_name]
    head.eval()
    with torch.no_grad():
        logits = head(emb_test_updated.to(device))
        preds  = logits.argmax(dim=1).cpu().tolist()

    acc          = accuracy_score(y_test, preds)
    micro_f1     = f1_score(y_test, preds, average="micro")
    macro_f1     = f1_score(y_test, preds, average="macro")
    weighted_f1  = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")

    print(f"Metrics after fine-tuning embeddings for {group_name}:")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")


=== Recomputing embeddings for group2 ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Metrics after fine-tuning embeddings for group2:
  Accuracy    :  60.31%
  Micro-F1    :  60.31%
  Macro-F1    :  59.28%
  Weighted-F1 :  59.81%
  Macro-Recall:  59.41%

=== Recomputing embeddings for group3 ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Metrics after fine-tuning embeddings for group3:
  Accuracy    :  58.44%
  Micro-F1    :  58.44%
  Macro-F1    :  57.35%
  Weighted-F1 :  58.20%
  Macro-Recall:  57.34%

=== Recomputing embeddings for group4 ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Metrics after fine-tuning embeddings for group4:
  Accuracy    :  53.12%
  Micro-F1    :  53.12%
  Macro-F1    :  51.04%
  Weighted-F1 :  52.18%
  Macro-Recall:  51.46%



In [10]:

model_1 = SentenceTransformer("intfloat/multilingual-e5-large-instruct")
model_1.to(device)

for p in model_1.parameters():
    p.requires_grad = False

transformer = model_1._modules['0']
hf_model    = transformer.auto_model
for p in hf_model.get_input_embeddings().parameters():
    p.requires_grad = True

blocks = None
for name, module in hf_model.named_modules():
    if isinstance(module, torch.nn.ModuleList):
        blocks = module
        print(f"→ Found transformer blocks under: {name}")
        break
if blocks is None:
    raise RuntimeError("Could not find transformer blocks")

for block in blocks[-3:]:
    for p in block.parameters():
        p.requires_grad = True

print(f"Un-froze {sum(p.requires_grad for p in model_1.parameters())} parameters total.\n")



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

→ Found transformer blocks under: encoder.layer
Un-froze 49 parameters total.



In [11]:

results = {}

for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"=== Tuning on {group_name} ===")
    df_all = pd.concat(labeled_group.values(), ignore_index=True)
    texts = df_all["title"].tolist()
    labels = df_all["label"].astype(int).tolist()
    X_train, X_test, y_train, y_test = train_test_split(
        texts, labels, test_size=0.2, random_state=42, stratify=labels
    )

    emb_train = model_1.encode(
        X_train, device=device, batch_size=16,
        convert_to_tensor=True, show_progress_bar=True
    )
    del X_train
    emb_test = model_1.encode(
        X_test, device=device, batch_size=16,
        convert_to_tensor=True, show_progress_bar=True
    )
    del X_test

    d = emb_train.size(1)
    head = nn.Sequential(
        nn.Linear(d, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    ).to(device)

    trainable = list(head.parameters()) + [
        p for p in model_1.parameters() if p.requires_grad
    ]
    optimizer = torch.optim.Adam(trainable, lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    train_ds = TensorDataset(emb_train, torch.tensor(y_train, device=device))
    train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)

    best_prev_loss = float("inf")
    tol = 1e-4
    patience = 3
    stale_epochs = 0
    max_epochs = 100

    for epoch in range(1, max_epochs + 1):
        head.train()
        epoch_loss = 0.0
        for emb_b, lab_b in train_dl:
            optimizer.zero_grad()
            logits = head(emb_b)
            loss = criterion(logits, lab_b)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * emb_b.size(0)
        avg_loss = epoch_loss / len(train_ds)
        print(f"  Epoch {epoch:>3} — Loss: {avg_loss:.6f}")
        improvement = best_prev_loss - avg_loss
        if improvement < tol:
            stale_epochs += 1
            print(f"  ↳ small improvement (Δ={improvement:.2e}) stale {stale_epochs}/{patience}")
        else:
            stale_epochs = 0
            best_prev_loss = avg_loss
        if stale_epochs >= patience:
            print(f"  ↳ Early stopping at epoch {epoch}")
            break

    head.eval()
    with torch.no_grad():
        logits = head(emb_test.to(device))
        preds = logits.argmax(dim=1).cpu().tolist()

    acc = accuracy_score(y_test, preds)
    micro_f1 = f1_score(y_test, preds, average="micro")
    macro_f1 = f1_score(y_test, preds, average="macro")
    weighted_f1 = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")

    print(f"Metrics for {group_name}:")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")

    results[group_name] = head



=== Tuning on group2 ===


Batches:   0%|          | 0/80 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

  Epoch   1 — Loss: 0.693612
  Epoch   2 — Loss: 0.689324
  Epoch   3 — Loss: 0.688075
  Epoch   4 — Loss: 0.687093
  Epoch   5 — Loss: 0.686177
  Epoch   6 — Loss: 0.685764
  Epoch   7 — Loss: 0.683765
  Epoch   8 — Loss: 0.682302
  Epoch   9 — Loss: 0.680985
  Epoch  10 — Loss: 0.680023
  Epoch  11 — Loss: 0.677966
  Epoch  12 — Loss: 0.676489
  Epoch  13 — Loss: 0.675168
  Epoch  14 — Loss: 0.673455
  Epoch  15 — Loss: 0.671443
  Epoch  16 — Loss: 0.670294
  Epoch  17 — Loss: 0.668089
  Epoch  18 — Loss: 0.666471
  Epoch  19 — Loss: 0.664802
  Epoch  20 — Loss: 0.663031
  Epoch  21 — Loss: 0.661499
  Epoch  22 — Loss: 0.660181
  Epoch  23 — Loss: 0.658979
  Epoch  24 — Loss: 0.657566
  Epoch  25 — Loss: 0.655376
  Epoch  26 — Loss: 0.654151
  Epoch  27 — Loss: 0.652940
  Epoch  28 — Loss: 0.651008
  Epoch  29 — Loss: 0.649806
  Epoch  30 — Loss: 0.649135
  Epoch  31 — Loss: 0.647601
  Epoch  32 — Loss: 0.647085
  Epoch  33 — Loss: 0.645406
  Epoch  34 — Loss: 0.645294
  Epoch  35 — 

...

  Epoch   1 — Loss: 0.687592
  Epoch   2 — Loss: 0.685561
  Epoch   3 — Loss: 0.684552
  Epoch   4 — Loss: 0.683831
  Epoch   5 — Loss: 0.683113
  Epoch   6 — Loss: 0.682567
  Epoch   7 — Loss: 0.681761
  Epoch   8 — Loss: 0.681027
  Epoch   9 — Loss: 0.680065
  Epoch  10 — Loss: 0.679377
  Epoch  11 — Loss: 0.678324
  Epoch  12 — Loss: 0.677723
  Epoch  13 — Loss: 0.676814
  Epoch  14 — Loss: 0.675840
  Epoch  15 — Loss: 0.674690
  Epoch  16 — Loss: 0.673258
  Epoch  17 — Loss: 0.672322
  Epoch  18 — Loss: 0.671264
  Epoch  19 — Loss: 0.670121
  Epoch  20 — Loss: 0.668504
  Epoch  21 — Loss: 0.667509
  Epoch  22 — Loss: 0.665938
  Epoch  23 — Loss: 0.665338
  Epoch  24 — Loss: 0.663546
  Epoch  25 — Loss: 0.663152
  Epoch  26 — Loss: 0.661455
  Epoch  27 — Loss: 0.659267
  Epoch  28 — Loss: 0.658394
  Epoch  29 — Loss: 0.656864
  Epoch  30 — Loss: 0.656133
  Epoch  31 — Loss: 0.654904
  Epoch  32 — Loss: 0.652934
  Epoch  33 — Loss: 0.651922
  Epoch  34 — Loss: 0.650366
  Epoch  35 — 

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

  Epoch   1 — Loss: 0.689470
  Epoch   2 — Loss: 0.687361
  Epoch   3 — Loss: 0.686586
  Epoch   4 — Loss: 0.685887
  Epoch   5 — Loss: 0.685318
  Epoch   6 — Loss: 0.684577
  Epoch   7 — Loss: 0.683731
  Epoch   8 — Loss: 0.682822
  Epoch   9 — Loss: 0.682252
  Epoch  10 — Loss: 0.681144
  Epoch  11 — Loss: 0.680427
  Epoch  12 — Loss: 0.679706
  Epoch  13 — Loss: 0.678462
  Epoch  14 — Loss: 0.677267
  Epoch  15 — Loss: 0.675957
  Epoch  16 — Loss: 0.675048
  Epoch  17 — Loss: 0.674182
  Epoch  18 — Loss: 0.672852
  Epoch  19 — Loss: 0.672648
  Epoch  20 — Loss: 0.670701
  Epoch  21 — Loss: 0.669128
  Epoch  22 — Loss: 0.667729
  Epoch  23 — Loss: 0.666627
  Epoch  24 — Loss: 0.665430
  Epoch  25 — Loss: 0.664121
  Epoch  26 — Loss: 0.662417
  Epoch  27 — Loss: 0.661937
  Epoch  28 — Loss: 0.660749
  Epoch  29 — Loss: 0.659288
  Epoch  30 — Loss: 0.658571
  Epoch  31 — Loss: 0.656871
  Epoch  32 — Loss: 0.655430
  Epoch  33 — Loss: 0.655025
  Epoch  34 — Loss: 0.653184
  Epoch  35 — 

In [12]:
# Re-evaluate using the fine-tuned embedding layers + trained heads
for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"=== Re-evaluating {group_name} with updated weights ===")
    df_all = pd.concat(labeled_group.values(), ignore_index=True)
    texts  = df_all["title"].tolist()
    labels = df_all["label"].astype(int).tolist()
    _, X_test, _, y_test = train_test_split(
        texts, labels,
        test_size=0.2,
        random_state=42,
        stratify=labels
    )
    emb_test_updated = model_1.encode(
        X_test,
        device=device,
        batch_size=16,
        convert_to_tensor=True,
        show_progress_bar=True
    )
    head = results[group_name]
    head.eval()
    with torch.no_grad():
        logits = head(emb_test_updated.to(device))
        preds  = logits.argmax(dim=1).cpu().tolist()
    acc          = accuracy_score(y_test, preds)
    micro_f1     = f1_score(y_test, preds, average="micro")
    macro_f1     = f1_score(y_test, preds, average="macro")
    weighted_f1  = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")




=== Re-evaluating group2 with updated weights ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  Accuracy    :  59.06%
  Micro-F1    :  59.06%
  Macro-F1    :  58.72%
  Weighted-F1 :  59.03%
  Macro-Recall:  58.71%

=== Re-evaluating group3 with updated weights ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  Accuracy    :  57.50%
  Micro-F1    :  57.50%
  Macro-F1    :  56.60%
  Weighted-F1 :  57.38%
  Macro-Recall:  56.59%

=== Re-evaluating group4 with updated weights ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  Accuracy    :  55.94%
  Micro-F1    :  55.94%
  Macro-F1    :  53.52%
  Weighted-F1 :  54.71%
  Macro-Recall:  54.06%



In [13]:
model_2 = SentenceTransformer(
    "Lajavaness/bilingual-embedding-large",
    trust_remote_code=True)

model_2.to(device)
for p in model_2.parameters():
    p.requires_grad = False

transformer = model_2._modules['0']
hf_model    = transformer.auto_model

for p in hf_model.get_input_embeddings().parameters():
    p.requires_grad = True

blocks = None
for name, module in hf_model.named_modules():
    if isinstance(module, torch.nn.ModuleList):
        blocks = module
        print(f"→ Found blocks under: {name}")
        break
if blocks is None:
    raise RuntimeError("Could not find transformer blocks in hf_model")

for block in blocks[-3:]:
    for p in block.parameters():
        p.requires_grad = True

print(f"Un-froze {sum(p.requires_grad for p in model_1.parameters())} params total.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config-checkpoint.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

→ Found blocks under: encoder.layer
Un-froze 49 params total.


In [15]:
results = {}

for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"=== Tuning on {group_name} ===")
    df_all = pd.concat(labeled_group.values(), ignore_index=True)
    texts = df_all["title"].tolist()
    labels = df_all["label"].astype(int).tolist()
    X_train, X_test, y_train, y_test = train_test_split(
        texts, labels, test_size=0.2, random_state=42, stratify=labels)

    emb_train = model_2.encode(
        X_train, device=device, batch_size=16,
        convert_to_tensor=True, show_progress_bar=True)

    del X_train
    emb_test = model_2.encode(
        X_test, device=device, batch_size=16,
        convert_to_tensor=True, show_progress_bar=True)

    del X_test
    d = emb_train.size(1)
    head = nn.Sequential(
        nn.Linear(d, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    ).to(device)
    trainable = list(head.parameters()) + [
        p for p in model_2.parameters() if p.requires_grad]

    optimizer = torch.optim.Adam(trainable, lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    train_ds = TensorDataset(emb_train, torch.tensor(y_train, device=device))
    train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)
    best_prev_loss = float("inf")
    tol = 1e-4
    patience = 3
    stale_epochs = 0
    max_epochs = 100
    for epoch in range(1, max_epochs + 1):
        head.train()
        epoch_loss = 0.0
        for emb_b, lab_b in train_dl:
            optimizer.zero_grad()
            logits = head(emb_b)
            loss = criterion(logits, lab_b)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * emb_b.size(0)
        avg_loss = epoch_loss / len(train_ds)
        print(f"  Epoch {epoch:>3} — Loss: {avg_loss:.6f}")
        improvement = best_prev_loss - avg_loss
        if improvement < tol:
            stale_epochs += 1
            print(f"  ↳ small improvement (Δ={improvement:.2e}) stale {stale_epochs}/{patience}")
        else:
            stale_epochs = 0
            best_prev_loss = avg_loss
        if stale_epochs >= patience:
            print(f"  ↳ Early stopping at epoch {epoch}")
            break
    head.eval()
    with torch.no_grad():
        logits = head(emb_test.to(device))
        preds = logits.argmax(dim=1).cpu().tolist()

    acc = accuracy_score(y_test, preds)
    micro_f1 = f1_score(y_test, preds, average="micro")
    macro_f1 = f1_score(y_test, preds, average="macro")
    weighted_f1 = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")
    print(f"Metrics for {group_name}:")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")
    results[group_name] = head



=== Tuning on group2 ===


Batches:   0%|          | 0/80 [00:00<?, ?it/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

  Epoch   1 — Loss: 0.689878
  Epoch   2 — Loss: 0.687140
  Epoch   3 — Loss: 0.683613
  Epoch   4 — Loss: 0.679142
  Epoch   5 — Loss: 0.673652
  Epoch   6 — Loss: 0.668305
  Epoch   7 — Loss: 0.663050
  Epoch   8 — Loss: 0.657952
  Epoch   9 — Loss: 0.653254
  Epoch  10 — Loss: 0.649072
  Epoch  11 — Loss: 0.645324
  Epoch  12 — Loss: 0.642181
  Epoch  13 — Loss: 0.638604
  Epoch  14 — Loss: 0.635826
  Epoch  15 — Loss: 0.632476
  Epoch  16 — Loss: 0.630324
  Epoch  17 — Loss: 0.627159
  Epoch  18 — Loss: 0.624544
  Epoch  19 — Loss: 0.622096
  Epoch  20 — Loss: 0.619103
  Epoch  21 — Loss: 0.616999
  Epoch  22 — Loss: 0.614578
  Epoch  23 — Loss: 0.611888
  Epoch  24 — Loss: 0.609303
  Epoch  25 — Loss: 0.607044
  Epoch  26 — Loss: 0.604737
  Epoch  27 — Loss: 0.602488
  Epoch  28 — Loss: 0.600218
  Epoch  29 — Loss: 0.598155
  Epoch  30 — Loss: 0.595365
  Epoch  31 — Loss: 0.593404
  Epoch  32 — Loss: 0.591196
  Epoch  33 — Loss: 0.588166
  Epoch  34 — Loss: 0.586441
  Epoch  35 — 

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

  Epoch   1 — Loss: 0.695775
  Epoch   2 — Loss: 0.689023
  Epoch   3 — Loss: 0.683282
  Epoch   4 — Loss: 0.679513
  Epoch   5 — Loss: 0.676008
  Epoch   6 — Loss: 0.672447
  Epoch   7 — Loss: 0.669211
  Epoch   8 — Loss: 0.665332
  Epoch   9 — Loss: 0.661666
  Epoch  10 — Loss: 0.657597
  Epoch  11 — Loss: 0.653778
  Epoch  12 — Loss: 0.649916
  Epoch  13 — Loss: 0.646380
  Epoch  14 — Loss: 0.642272
  Epoch  15 — Loss: 0.638513
  Epoch  16 — Loss: 0.635046
  Epoch  17 — Loss: 0.631257
  Epoch  18 — Loss: 0.627360
  Epoch  19 — Loss: 0.623805
  Epoch  20 — Loss: 0.620497
  Epoch  21 — Loss: 0.616496
  Epoch  22 — Loss: 0.613360
  Epoch  23 — Loss: 0.610199
  Epoch  24 — Loss: 0.606809
  Epoch  25 — Loss: 0.603498
  Epoch  26 — Loss: 0.600441
  Epoch  27 — Loss: 0.597191
  Epoch  28 — Loss: 0.593932
  Epoch  29 — Loss: 0.591321
  Epoch  30 — Loss: 0.588136
  Epoch  31 — Loss: 0.584800
  Epoch  32 — Loss: 0.581771
  Epoch  33 — Loss: 0.579256
  Epoch  34 — Loss: 0.575928
  Epoch  35 — 

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

  Epoch   1 — Loss: 0.688638
  Epoch   2 — Loss: 0.685498
  Epoch   3 — Loss: 0.682630
  Epoch   4 — Loss: 0.679024
  Epoch   5 — Loss: 0.674858
  Epoch   6 — Loss: 0.670322
  Epoch   7 — Loss: 0.666086
  Epoch   8 — Loss: 0.661535
  Epoch   9 — Loss: 0.657152
  Epoch  10 — Loss: 0.652488
  Epoch  11 — Loss: 0.649171
  Epoch  12 — Loss: 0.644251
  Epoch  13 — Loss: 0.640496
  Epoch  14 — Loss: 0.637168
  Epoch  15 — Loss: 0.633441
  Epoch  16 — Loss: 0.630276
  Epoch  17 — Loss: 0.627018
  Epoch  18 — Loss: 0.623191
  Epoch  19 — Loss: 0.620272
  Epoch  20 — Loss: 0.617175
  Epoch  21 — Loss: 0.614618
  Epoch  22 — Loss: 0.611340
  Epoch  23 — Loss: 0.608489
  Epoch  24 — Loss: 0.605613
  Epoch  25 — Loss: 0.603171
  Epoch  26 — Loss: 0.600409
  Epoch  27 — Loss: 0.597495
  Epoch  28 — Loss: 0.594707
  Epoch  29 — Loss: 0.592335
  Epoch  30 — Loss: 0.590396
  Epoch  31 — Loss: 0.587503
  Epoch  32 — Loss: 0.584803
  Epoch  33 — Loss: 0.582461
  Epoch  34 — Loss: 0.580247
  Epoch  35 — 

In [17]:
# Re-evaluate with updated embedding and head weights
for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"=== Re-evaluating {group_name} ===")
    df_all = pd.concat(labeled_group.values(), ignore_index=True)
    texts  = df_all["title"].tolist()
    labels = df_all["label"].astype(int).tolist()
    _, X_test, _, y_test = train_test_split(
        texts, labels,
        test_size=0.2,
        random_state=42,
        stratify=labels
    )
    emb_test_updated = model_2.encode(
        X_test,
        device=device,
        batch_size=16,
        convert_to_tensor=True,
        show_progress_bar=True
    )
    head = results[group_name]
    head.eval()
    with torch.no_grad():
        logits = head(emb_test_updated.to(device))
        preds  = logits.argmax(dim=1).cpu().tolist()
    acc          = accuracy_score(y_test, preds)
    micro_f1     = f1_score(y_test, preds, average="micro")
    macro_f1     = f1_score(y_test, preds, average="macro")
    weighted_f1  = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")
    print(f"Metrics for {group_name}:")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")


=== Re-evaluating group2 ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Metrics for group2:
  Accuracy    :  58.44%
  Micro-F1    :  58.44%
  Macro-F1    :  57.74%
  Weighted-F1 :  58.18%
  Macro-Recall:  57.78%

=== Re-evaluating group3 ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Metrics for group3:
  Accuracy    :  57.50%
  Micro-F1    :  57.50%
  Macro-F1    :  57.17%
  Weighted-F1 :  57.64%
  Macro-Recall:  57.30%

=== Re-evaluating group4 ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Metrics for group4:
  Accuracy    :  52.81%
  Micro-F1    :  52.81%
  Macro-F1    :  52.02%
  Weighted-F1 :  52.72%
  Macro-Recall:  52.03%



In [16]:
model_3 = SentenceTransformer(
    "sentence-transformers/stsb-roberta-large",
    trust_remote_code=True)

for p in model_3.parameters():
    p.requires_grad = False

transformer = model_3._modules['0']
hf_model    = transformer.auto_model

for p in hf_model.get_input_embeddings().parameters():
    p.requires_grad = True

blocks = None
for name, module in hf_model.named_modules():
    if isinstance(module, torch.nn.ModuleList):
        blocks = module
        print(f"→ Found blocks under: {name}")
        break
if blocks is None:
    raise RuntimeError("Could not find transformer blocks in hf_model")

for block in blocks[-3:]:
    for p in block.parameters():
        p.requires_grad = True

print(f"Un-froze {sum(p.requires_grad for p in model_1.parameters())} params total.")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

→ Found blocks under: encoder.layer
Un-froze 49 params total.


In [18]:
results = {}

for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"=== Tuning on {group_name} ===")
    df_all = pd.concat(labeled_group.values(), ignore_index=True)
    texts = df_all["title"].tolist()
    labels = df_all["label"].astype(int).tolist()
    X_train, X_test, y_train, y_test = train_test_split(
        texts, labels, test_size=0.2, random_state=42, stratify=labels)

    emb_train = model_3.encode(
        X_train, device=device, batch_size=16,
        convert_to_tensor=True, show_progress_bar=True)
    del X_train

    emb_test = model_3.encode(
        X_test, device=device, batch_size=16,
        convert_to_tensor=True, show_progress_bar=True)
    del X_test

    d = emb_train.size(1)
    head = nn.Sequential(
        nn.Linear(d, 64),
        nn.ReLU(),
        nn.Linear(64, 2)).to(device)

    trainable = list(head.parameters()) + [
        p for p in model_3.parameters() if p.requires_grad]

    optimizer = torch.optim.Adam(trainable, lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    train_ds = TensorDataset(emb_train, torch.tensor(y_train, device=device))
    train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)

    best_prev_loss = float("inf")
    tol = 1e-4
    patience = 3
    stale_epochs = 0
    max_epochs = 100

    for epoch in range(1, max_epochs + 1):
        head.train()
        epoch_loss = 0.0
        for emb_b, lab_b in train_dl:
            optimizer.zero_grad()
            logits = head(emb_b)
            loss = criterion(logits, lab_b)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * emb_b.size(0)
        avg_loss = epoch_loss / len(train_ds)
        print(f"  Epoch {epoch:>3} — Loss: {avg_loss:.6f}")
        improvement = best_prev_loss - avg_loss
        if improvement < tol:
            stale_epochs += 1
            print(f"  ↳ small improvement (Δ={improvement:.2e}) stale {stale_epochs}/{patience}")
        else:
            stale_epochs = 0
            best_prev_loss = avg_loss
        if stale_epochs >= patience:
            print(f"  ↳ Early stopping at epoch {epoch}")
            break

    head.eval()
    with torch.no_grad():
        logits = head(emb_test.to(device))
        preds = logits.argmax(dim=1).cpu().tolist()

    acc = accuracy_score(y_test, preds)
    micro_f1 = f1_score(y_test, preds, average="micro")
    macro_f1 = f1_score(y_test, preds, average="macro")
    weighted_f1 = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")

    print(f"Metrics for {group_name}:")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")

    results[group_name] = head


=== Tuning on group2 ===


Batches:   0%|          | 0/80 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

  Epoch   1 — Loss: 0.684203
  Epoch   2 — Loss: 0.651914
  Epoch   3 — Loss: 0.634159
  Epoch   4 — Loss: 0.616365
  Epoch   5 — Loss: 0.602416
  Epoch   6 — Loss: 0.585122
  Epoch   7 — Loss: 0.568690
  Epoch   8 — Loss: 0.552730
  Epoch   9 — Loss: 0.539183
  Epoch  10 — Loss: 0.522088
  Epoch  11 — Loss: 0.505964
  Epoch  12 — Loss: 0.490110
  Epoch  13 — Loss: 0.473711
  Epoch  14 — Loss: 0.459154
  Epoch  15 — Loss: 0.442652
  Epoch  16 — Loss: 0.426321
  Epoch  17 — Loss: 0.414032
  Epoch  18 — Loss: 0.397117
  Epoch  19 — Loss: 0.384131
  Epoch  20 — Loss: 0.372073
  Epoch  21 — Loss: 0.359825
  Epoch  22 — Loss: 0.348472
  Epoch  23 — Loss: 0.333871
  Epoch  24 — Loss: 0.325948
  Epoch  25 — Loss: 0.312595
  Epoch  26 — Loss: 0.303959
  Epoch  27 — Loss: 0.292465
  Epoch  28 — Loss: 0.281778
  Epoch  29 — Loss: 0.272511
  Epoch  30 — Loss: 0.265515
  Epoch  31 — Loss: 0.255212
  Epoch  32 — Loss: 0.247497
  Epoch  33 — Loss: 0.239874
  Epoch  34 — Loss: 0.232152
  Epoch  35 — 

...

  Epoch   1 — Loss: 0.680906
  Epoch   2 — Loss: 0.651838
  Epoch   3 — Loss: 0.632582
  Epoch   4 — Loss: 0.613767
  Epoch   5 — Loss: 0.596141
  Epoch   6 — Loss: 0.579071
  Epoch   7 — Loss: 0.561556
  Epoch   8 — Loss: 0.541865
  Epoch   9 — Loss: 0.525246
  Epoch  10 — Loss: 0.506970
  Epoch  11 — Loss: 0.485707
  Epoch  12 — Loss: 0.469680
  Epoch  13 — Loss: 0.450482
  Epoch  14 — Loss: 0.433289
  Epoch  15 — Loss: 0.416671
  Epoch  16 — Loss: 0.399856
  Epoch  17 — Loss: 0.382653
  Epoch  18 — Loss: 0.367167
  Epoch  19 — Loss: 0.354555
  Epoch  20 — Loss: 0.338210
  Epoch  21 — Loss: 0.324430
  Epoch  22 — Loss: 0.309983
  Epoch  23 — Loss: 0.300754
  Epoch  24 — Loss: 0.285679
  Epoch  25 — Loss: 0.272687
  Epoch  26 — Loss: 0.264264
  Epoch  27 — Loss: 0.253202
  Epoch  28 — Loss: 0.242305
  Epoch  29 — Loss: 0.233397
  Epoch  30 — Loss: 0.225520
  Epoch  31 — Loss: 0.216467
  Epoch  32 — Loss: 0.208381
  Epoch  33 — Loss: 0.200821
  Epoch  34 — Loss: 0.193841
  Epoch  35 — 

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

  Epoch   1 — Loss: 0.686065
  Epoch   2 — Loss: 0.657601
  Epoch   3 — Loss: 0.638733
  Epoch   4 — Loss: 0.622019
  Epoch   5 — Loss: 0.604479
  Epoch   6 — Loss: 0.587428
  Epoch   7 — Loss: 0.571560
  Epoch   8 — Loss: 0.555145
  Epoch   9 — Loss: 0.536093
  Epoch  10 — Loss: 0.518895
  Epoch  11 — Loss: 0.501014
  Epoch  12 — Loss: 0.481923
  Epoch  13 — Loss: 0.466531
  Epoch  14 — Loss: 0.447624
  Epoch  15 — Loss: 0.430480
  Epoch  16 — Loss: 0.411713
  Epoch  17 — Loss: 0.395971
  Epoch  18 — Loss: 0.377812
  Epoch  19 — Loss: 0.364514
  Epoch  20 — Loss: 0.347185
  Epoch  21 — Loss: 0.331965
  Epoch  22 — Loss: 0.315839
  Epoch  23 — Loss: 0.303425
  Epoch  24 — Loss: 0.288428
  Epoch  25 — Loss: 0.277847
  Epoch  26 — Loss: 0.263011
  Epoch  27 — Loss: 0.252525
  Epoch  28 — Loss: 0.241455
  Epoch  29 — Loss: 0.229429
  Epoch  30 — Loss: 0.220189
  Epoch  31 — Loss: 0.208129
  Epoch  32 — Loss: 0.200499
  Epoch  33 — Loss: 0.191533
  Epoch  34 — Loss: 0.182055
  Epoch  35 — 

In [19]:
# Re-evaluate with updated embedding and head weights
for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"=== Re-evaluating {group_name} ===")
    df_all = pd.concat(labeled_group.values(), ignore_index=True)
    texts  = df_all["title"].tolist()
    labels = df_all["label"].astype(int).tolist()
    _, X_test, _, y_test = train_test_split(
        texts, labels,
        test_size=0.2,
        random_state=42,
        stratify=labels
    )
    emb_test_updated = model_3.encode(
        X_test,
        device=device,
        batch_size=16,
        convert_to_tensor=True,
        show_progress_bar=True
    )
    head = results[group_name]
    head.eval()
    with torch.no_grad():
        logits = head(emb_test_updated.to(device))
        preds  = logits.argmax(dim=1).cpu().tolist()
    acc          = accuracy_score(y_test, preds)
    micro_f1     = f1_score(y_test, preds, average="micro")
    macro_f1     = f1_score(y_test, preds, average="macro")
    weighted_f1  = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")
    print(f"Metrics for {group_name}:")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")


=== Re-evaluating group2 ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Metrics for group2:
  Accuracy    :  58.44%
  Micro-F1    :  58.44%
  Macro-F1    :  58.04%
  Weighted-F1 :  58.37%
  Macro-Recall:  58.03%

=== Re-evaluating group3 ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Metrics for group3:
  Accuracy    :  55.94%
  Micro-F1    :  55.94%
  Macro-F1    :  55.20%
  Weighted-F1 :  55.92%
  Macro-Recall:  55.20%

=== Re-evaluating group4 ===


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Metrics for group4:
  Accuracy    :  53.44%
  Micro-F1    :  53.44%
  Macro-F1    :  52.81%
  Weighted-F1 :  53.42%
  Macro-Recall:  52.81%

